In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import random,time
from random import randint
import re
import urllib
import sys
from urllib import request
from datetime import datetime
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime

def create_mkdir():
    ####建立資料夾
    mydir='cookpad'
    mydir_save_img="cookpad_save_img"
    mydir_save_log='cookpad_log'
    mydir_save_title_log='cookpad_title_log'
    
    if os.path.exists(mydir):
        print('mrdir name cookpad has been create')
    else:
        os.mkdir(mydir)

    if os.path.exists(mydir_save_img):
        print("mrdir name cookpad_save_img has been create")
    else:
        os.mkdir(mydir_save_img)
    
    if os.path.exists(mydir_save_log):
        print("mrdir name cookpad_save_log has been create")
    else:
        os.mkdir(mydir_save_log)
    if os.path.exists(mydir_save_title_log):
        print('mydir name cookpad_title_log has been create')
    else:
        os.mkdir(mydir_save_title_log)
#######

#####寫入檔案相關
def write_to_file(write_file_list):
    with open('./cookpad/'+save_name+'.txt','w',encoding="utf-8") as es:
        es.write('食譜名稱:{}'.format(title_name)+'|'+'\n')
        es.write('作者:{}'.format(arthor)+'|'+'\n')
        es.write('圖片:{}'.format(img)+'|'+'\n')
        es.write('份數:{}'.format(Number_of_copies)+'|'+'\n')
        es.write('料理時間:{}'.format(cook_time)+'|'+'\n')
        es.write('食材:{}'.format(set_food)+'|'+'\n')
        es.write('食材數量:{}'.format(set_food_count)+'|'+'\n')
        es.write('料理步驟:{}'.format(set_step)+'|'+'\n')
        print('寫檔完成')

def save_img(img):
    if img!="na":
        print("進入寫入圖片前")
        request.urlretrieve(img, './cookpad_save_img/{}.jpg'.format(save_name))#寫入圖片
        print("圖片下載完成")
    else:
        print("image not found")
def save_log(set_next_url):
    with open('./cookpad_log/'+str(set_next_url)+'_log'+'.txt','w',encoding="utf-8") as es2:
        es2.write(getnextpageurl)
    
def save_title_log(set_title_log_url):
    with open('./cookpad_title_log/'+str(set_title_log_url)+"_title_log"+'.txt','w',encoding='utf-8')as es3:
        es3.write(title_url)

def list_to_csv(list_save):
    data=DataFrame(list_save)#这时候是以行为标准写入的
    print(data)
    now = datetime.now()
    s1 = datetime.strftime(now,'%Y-%m-%d')
    s2= datetime.strftime(now,'%H:%M:%S')
    s2=s2.replace(":","_")
    #print (type(now))
    #print (now)
    #print (s1)
    #print(s2)
    save_file_data_time=s1+"_"+s2
    #print(s.replace(" ","")))
    data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')        

######## 


####文章控制相關
def count_time():
    time_strat=time.time()
    #time.sleep(5)
    time.sleep(randint(5,15))
    time_end=time.time()
    print(time_end-time_strat)

def get_main_body(url):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'}
    ss=requests.session()
    res=ss.get(url,headers=headers)
    cookies=ss.cookies.get_dict()
    #print(ss.cookies.get_dict())

    res=ss.get(url,headers=headers,cookies=cookies)
    soup=BeautifulSoup(res.text,'html.parser')
    return soup

#食材,食材數量,步驟

def setfood_and_set_food_count_step(food_test):
    set_food=''
    set_step=''
    set_food_count=''
    step_number=1
    for x in food_test:
        get_food=x.select('div')[0].text
        #print(get_food)
        check_len=len(get_food.split(' '))
        #print(check_len)
        try:
            get_food=get_food.split(' ')[8]
            set_food=set_food+get_food+' '
        except(IndexError):
            print('get_food陣列沒有第8個值')
        #print(get_food)
        get_food_count=x.select('div')[0].select('bdi')[0].text
    
        set_food_count=set_food_count+get_food_count+" "
    for z in step:
        get_step=z.text
        set_step=set_step+str(step_number)+"."+get_step
        step_number=step_number+1

    return set_food, set_food_count  ,set_step      

    
def check_title_log_url(title_url):
    count_wrong=5
    save_list=[]   
    #建立檢查的list
    #test='./cookpad_title_log_check/1_泰式檸檬魚簡易版.txt'
    test_list=os.listdir('./cookpad_title_log/')
    #print(test_list)
    for i in test_list:
        with open ('./cookpad_title_log/'+i,'r',encoding='utf-8')as es4:
            data=es4.read()
        #print(data)
            save_list.append(data)
    
    for g in save_list:
        if g==title_url:
           count_wrong=6
     
           
    if count_wrong==6:
        print("這個url已經重複了,程式不寫入檔案" )
        return count_wrong
    else:
        print("這個url沒有重複,程式繼續")
        return count_wrong

######

create_mkdir()

###一些計數器變數    
url="https://cookpad.com/tw"
set_next_url_count=1
set_next_url=1     #nextlogurl編號
set_total_items=3550 #檔案編號
set_while_stop=1
list_save=[['食譜名稱', '作者','圖片','份數', '料理時間', '食材','食材數量', '料理步驟']]
tep_list=[]
save_next_html=[]#存下檢查爬到哪裡了
set_food=''
set_step=''
log_list_save=[['編號','網站']]
######

while (set_while_stop==1):
#for go  in  range(1,3):
    get_main_body(url)    
    soup=get_main_body(url)
    #print(soup)    
    mainbody=soup.select('div.media')
    getnextpageurl="https://cookpad.com"+soup.select('div#feed_pagination')[0].select('a')[0]['href']
    save_next_html.append(getnextpageurl)#紀錄html
    

    for i in mainbody:
        
        print("第{}筆資料".format(set_total_items))
        title=i.select('div.media__body')[0].select('a.flex-grow')[0]['aria-label']
        test2=i.select('div.media__body')[0].select('a.flex-grow')[0]
        title_url="https://cookpad.com"+i.select('div.media__body')[0].select('a.flex-grow')[0]['href']
        url=title_url
        get_main_body(url)
        soup=get_main_body(url)
        #print(soup)
    
        print(title)
        #print(test2)
        print(title_url)
        
        count_time()
        #如果img沒有抓到
        img=soup.select('div.tofu_image')
        if len(img)<=0:
            print("img is not found")
            img="na"
        else:
            img=img[0].select('img')[0]['src']
            split_img=img.split(':')[1]#進行切片防止:被轉碼
            test=urllib.parse.quote(split_img)#網址中文轉碼
            img="https:"+test#進行網址重組
        title_name=soup.select('section.intro-container')[0].select('h1.break-words')[0].text
        #artice=soup.select('div.recipe-show__story')[0].select('p.mb-2')[0].text.split('#')[0]
        
        arthor=soup.select('section.document__section.media.author-container')[0].select('span.text-cookpad-16')[0].text
        cook_time=soup.select('div.p-3.border-t.border-quaternary.text-center.text-secondary')
        if len(cook_time) <= 0 :
            cook_time = "Na"
            print("cook_tine is na")
        else:
            cook_time = cook_time[0].select('span.px-4')[0].text #有些資料沒有
           
        food_test=soup.select('div.ingredient-list')[0].select('ol.list-unstyled.text-cookpad-18')[0].select('li')
        Number_of_copies=soup.select('div.text-secondary.mt-2')[0].text
        if len(Number_of_copies.split())<=0:
            Number_of_copies='na'
        else:
            print('Number_of_copies is alive')
    
            
        step=soup.select('ol.numbered-list')[0].select('p.mb-2')
            
        set_food, set_food_count ,set_step= setfood_and_set_food_count_step(food_test)   
       
        
        step_number=1#重設步驟變數
        #food_total_and_food_step(food_test,step,step_number)
        
        #print(img)
        #print(title_name)
        #print(artice)
        #print(Awards)
        #print(arthor)
        #print(cook_time)
        #print(food_total)
        #print(set_food)
        #print(set_step)
        #print(img)
        #print(Number_of_copies)
        #存入資料
        title_name=title_name.replace(" ","")
        set_food=set_food.replace(" ","")
            
        tep_list.append(title_name)
        tep_list.append(arthor)
        tep_list.append(img)
        tep_list.append(Number_of_copies)
        tep_list.append(cook_time)
        tep_list.append(set_food)
        tep_list.append(set_food_count)
        tep_list.append(set_step)
        list_save.append(tep_list)
        
        #print(list_save)
        tep_list=[]
        # write_to_file(img, content, cook, cook_time)
           
        #print(list_save)
        #print("_____________________________________")
            
        #print("正規化檔名前")
        #print("_____________________________________")
        string=title_name
        re_title_name=re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",string)
        #print(re_title_name)
        save_name_id=str(set_total_items)
        save_name=save_name_id+"_"+re_title_name
        
        count_wrong=check_title_log_url(title_url)
        
        if count_wrong!=6:
            print("程式繼續")
        else:
            set_total_items+=1
            print("程式不寫檔")
            continue
            
        
        
        #print(save_name)
        #print("正規化檔名後")
            
            
            
            
        #print("_____________________________________")
        
        #print(入寫檔前')
        
        write_file_list=[title_name, arthor, img, Number_of_copies,cook_time,set_food,set_food_count,set_step]
        write_to_file(write_file_list)
        save_title_log(save_name)    
        save_img(img)    
 
                
           
            
        print("_____________________________________")
            
        
      
        set_total_items+=1

    #寫入next_URL_LOG       
    save_log(set_next_url)
    set_next_url+=1
    
    url=getnextpageurl
    print("+++++++換頁+++++++++++++++++")
    print("第{}個頁面".format(set_next_url_count))
    
    print(getnextpageurl)
    set_next_url_count+=1
"""
    if set_total_items==1001:
        set_while_stop=set_while_stop-1
        print("已經有一千筆資料了")  
"""    

if getnextpageurl=='https://cookpad.com/tw?after=1526305209.0':
    set_while_stop=set_while_stop-1
    
    

print(set_total_items-1)

list_to_csv(list_save)




mrdir name cookpad has been create
mrdir name cookpad_save_img has been create
mrdir name cookpad_save_log has been create
mydir name cookpad_title_log has been create
第3550筆資料
自製花枝煎 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12962330-%E8%87%AA%E8%A3%BD%E8%8A%B1%E6%9E%9D%E7%85%8E
5.000109910964966
cook_tine is na
這個url沒有重複,程式繼續
程式繼續
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第3551筆資料
南瓜堅果沙拉 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12887090-%E5%8D%97%E7%93%9C%E5%A0%85%E6%9E%9C%E6%B2%99%E6%8B%89
11.000959396362305
cook_tine is na
Number_of_copies is alive
這個url沒有重複,程式繼續
程式繼續
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第3552筆資料
青醬奶油海鮮義大利麵 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12924076-%E9%9D%92%E9%86%AC%E5%A5%B6%E6%B2%B9%E6%B5%B7%E9%AE%AE%E7%BE%A9%E5%A4%A7%E5%88%A9%E9%BA%B5
14.000673770904541
cook_tine is na
這個url沒有重複,程式繼續
程式繼續
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第3553筆資料
蒜香四季豆 link
https://cookpad.com/tw/%E9%A3%9F%E8%A

In [ ]:
#print(save_next_html)
print(len(save_next_html))




In [ ]:
#print(list_save)
print(len(list_save))
print(list_save[990])



In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime
#包含两个不同的子列表[1,2,3,4]和[5,6,7,8]
data=DataFrame(list_save)#这时候是以行为标准写入的
print(data)

now = datetime.now()
s1 = datetime.strftime(now,'%Y-%m-%d')
s2= datetime.strftime(now,'%H:%M:%S')
s2=s2.replace(":","_")
#print (type(now))
#print (now)
print (s1)
print(s2)
save_file_data_time=s1+"_"+s2
#print(s.replace(" ","")))

data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')

In [ ]:
!pip install pandas

In [ ]:
def check_getnexturl(url):
    check=url
    with open  